In [17]:
# CARGAMOS LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

import boto3
import s3fs
import fastparquet
import awswrangler as wr
import os

import pyarrow.parquet as pq
import pyarrow as pa

import requests
import json
from io import BytesIO
from pandas.tseries.offsets import BDay

import math

In [18]:
# INICIALIZAMOS UNA SESION EN AWS
session = boto3.Session(
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
    region_name=os.environ['S3_REGION'])

s3_client = boto3.client('s3')
ibex_historical_data = {}
bucket_name = 'stock-market-historical-data'
raw_folder = 'raw/yfinance/'
pre_folder = 'preprocessed/yfinance/'
pro_folder = 'processed/yfinance/'

OBJETIVO DE ESTE NOTEBOOK:
- DESCARGAR DE DISTINTOS ORIGENES LOS DATOS DE LOS ACTIVOS DEL IBEX35
- COMPARAR LOS DATOS OBTENIDOS 

## CARGA DE COMPOSICIÓN HISTÓRICA DE IBEX35

In [19]:
# LEEMOS EL FICHERO CON LA COMPOSICIÓN HISTORICA
ibex_constituents = pd.read_excel('data_processing/Historical Composition IBEX35.xlsx')
ibex_constituents.head()

,Activo,Inclusion,Exclusion
0,ACE,1991-01-02,2003-06-02
1,ACX,1991-01-02,1991-07-01
2,ALB,1991-01-02,1991-07-01
3,ASL,1991-01-02,1994-01-03
4,BBV,1991-01-02,2000-01-31


In [20]:
# ibex_constituents['Exclusion'] -= BDay(1)

# ibex_constituents['Inclusion'] = ibex_constituents['Inclusion'].dt.date
# ibex_constituents['Exclusion'] = ibex_constituents['Exclusion'].dt.date

# OBTENEMOS EL DIA ACTUAL
today = datetime.date.today().strftime('%Y-%m-%d')

# COMPLETAMOS LOS DATOS NULOS EN EL FICHERO DE COMPOSICIÓN HISTORICA CON LA FECHA ACTUAL
ibex_constituents.fillna(today,inplace=True)

# DEFINIMOS UNA VARIABLE PARA TODOS LOS ACTIVOS
activos = ibex_constituents.Activo
ibex_constituents['Ticker'] = activos.apply(lambda x: x.split('_')[0])

unique_activos = list(set(activos))

In [21]:
current_constituents = ibex_constituents.loc[ibex_constituents['Exclusion'] == today]
current_constituents

,Activo,Inclusion,Exclusion,Ticker
5,BKT,1991-01-02,2023-08-27,BKT
18,IBE,1991-01-02,2023-08-27,IBE
21,REP,1991-01-02,2023-08-27,REP
25,TEF,1991-01-02,2023-08-27,TEF
71,ACS,1998-04-02,2023-08-27,ACS
79,FER,1999-07-01,2023-08-27,FER
80,IDR,1999-07-01,2023-08-27,IDR
85,BBVA,2000-01-31,2023-08-27,BBVA
95,ITX,2001-07-02,2023-08-27,ITX
96,SAN_1,2001-11-01,2023-08-27,SAN


### DESCARGAMOS LOS DATOS DE YAHOO FINANCE

In [22]:
def upload_parquet_to_s3(bucket_name, file_name, folder_name, dataframe):
    full_path = folder_name + file_name + '.parquet'
    s3 = boto3.resource('s3')
    table = pa.Table.from_pandas(dataframe)
    buf = BytesIO()
    pq.write_table(table, buf)
    s3.Object(bucket_name, full_path).put(Body=buf.getvalue())

In [48]:
ibex_constituents[ibex_constituents['Activo']=='SOL'].index[0]

64

In [50]:
ibex_constituents.Exclusion[i]

Timestamp('2003-01-02 00:00:00')

In [37]:
# REALIZAMOS LA DESCARGA DE LOS DATOS DE YFINANCE Y LOS CARGAMOS EN UN BUCKET DE S3

failed_loads = []
i = ibex_constituents[ibex_constituents['Activo']=='BBV'].index[0]

ticker = 'BBVA' + '.MC'
print(i)
file_name = ibex_constituents.Activo[i] +'.parquet'
start_date = ibex_constituents.Inclusion[i]
end_date = ibex_constituents.Exclusion[i]
full_path = preprocessed_folder + file_name

stock = yf.download(ticker, start=start_date, end=end_date)
print(stock)
stock = pd.DataFrame(stock)

table = pa.Table.from_pandas(stock)
buf = BytesIO()
pq.write_table(table, buf)

s3.Object(bucket_name, full_path).put(Body=buf.getvalue())
print(ibex_constituents.Activo[i])


4
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2000-01-03  13.690602  13.757854  13.594527  13.623349   4.755257   8244257
2000-01-04  13.450416  13.536882  13.219837  13.267874   4.631179   8522096
2000-01-05  13.142977  13.210230  12.912399  12.970044   4.527219  12159826
2000-01-06  12.970044  12.970044  12.970044  12.970044   4.527219         0
2000-01-07  13.248659  13.248659  12.998866  13.133370   4.584228  62261944
2000-01-10  13.277482  13.277482  12.902791  12.922006   4.510450  58740351
2000-01-11  12.873969  12.893184  12.537709  12.662605   4.419908   6761292
2000-01-12  12.643391  12.643391  12.393597  12.518494   4.405469   8266754
2000-01-13  12.489672  12.537709  12.105374  12.211056   4.297278   9372669
2000-01-14  12.297523  12.480064  12.230271  12.451242   4.381802   8538333
2000-01-17  12.53

In [25]:
stock_prices = pq.read_table(download_s3_parquet_file(s3, bucket_name,'preprocessed/yfinance/SOL.parquet')).to_pandas()
stock_prices.index = pd.to_datetime(stock_prices.index).strftime('%Y-%m-%d')
negative_adj_close = stock_prices[stock_prices['Adj Close']<0]
if negative_adj_close.empty:
    close_price = stock_prices['Adj Close']
else:
    close_price = stock_prices['Close']

close_price.index = pd.to_datetime(close_price.index).strftime('%Y-%m-%d')
close_price

Date
2000-01-03   -12.893392
2000-01-04   -12.997465
2000-01-05   -12.488667
2000-01-07   -12.893392
2000-01-10   -13.032155
                ...    
2002-12-19     3.502554
2002-12-20     3.571063
2002-12-23     3.571063
2002-12-27     3.451172
2002-12-30     3.228515
Name: Adj Close, Length: 743, dtype: float64

In [33]:
raw_path = bucket_name + pre_folder
objects_dict = s3_client.list_objects_v2(
        Bucket=bucket_name,
        Prefix =pre_folder)
pre_filepaths = [item['Key'] for item in objects_dict['Contents'] if item['Key'].endswith('.parquet')]
act_list = [i.split('/')[-1].replace('.parquet','') for i in raw_filepaths]

In [7]:
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name

In [8]:
def download_s3_parquet_file(s3, bucket, key):
    buffer = BytesIO()
    s3.Object(bucket, key).download_fileobj(buffer)
    return buffer

In [9]:
bucket_name = 'stock-market-historical-data'
pre_folder = 'preprocessed/yfinance/'
pre_path = bucket_name + pre_folder
objects_dict = s3_client.list_objects_v2(
        Bucket=bucket_name,
        Prefix =pre_folder)
raw_filepaths = [item['Key'] for item in objects_dict['Contents'] if item['Key'].endswith('.parquet')]
act_list = [i.split('/')[-1].replace('.parquet','') for i in raw_filepaths]
current_constituents_filepath = [pre_folder + act + '.parquet' for act in current_constituents.Activo]

In [15]:
# REALIZAMOS LA DESCARGA DE LOS DATOS DE MARKETSTACK Y LOS CARGAMOS EN UN BUCKET DE S3
bucket_name = 'stock-market-historical-data'
raw_folder = 'preprocessed/yfinance/'
s3 = boto3.resource('s3')
failed_loads = []
for i in range(1,len(current_constituents_filepath)):
    act_data = pq.read_table(download_s3_parquet_file(s3, bucket_name,current_constituents_filepath[i])).to_pandas()
    act_data.index = pd.to_datetime(act_data.index)
    act_data.sort_index(inplace=True)
    date = act_data.index[-1] + datetime.timedelta(days=1)


    new_data = yf.download(current_constituents.Ticker.iloc[i]+'.MC', start=date)
    new_data.index = pd.to_datetime(new_data.index)

    act_data = pd.DataFrame(act_data)
    act_data = pd.concat([act_data,new_data], axis=0)
    act_data.sort_index(inplace=True)

    table = pa.Table.from_pandas(act_data)
    buf = BytesIO()
    pq.write_table(table, buf)

    s3.Object(bucket_name, current_constituents_filepath[i]).put(Body=buf.getvalue())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [23]:
# CARGAMOS EL DATAFRAME CON TODOS LOS DATOS DE CIERRE DE TODOS LOS ACTIVOS DESDE 02/01/1991
for i in range(len(raw_filepaths)):
    
    try:
        stock_prices = pq.read_table(download_s3_parquet_file(s3, bucket_name,raw_filepaths[i])).to_pandas()
        stock_prices.index = pd.to_datetime(stock_prices.index).strftime('%Y-%m-%d')
        returns = np.log(stock_prices['Adj Close']).diff()[1:]
        min_returns = returns.min().min()
        
        negative_adj_close = stock_prices[stock_prices['Adj Close']<0]
        print(negative_adj_close)
        if (negative_adj_close.empty):# | (min_returns > -1):
            close_price = stock_prices['Adj Close']
        else:
            close_price = stock_prices['Close']
            print(raw_filepaths[i])

        close_price.index = pd.to_datetime(close_price.index).strftime('%Y-%m-%d')
        ibex_historical_data[act_list[i]] = close_price
    except:
        ibex_historical_data[act_list[i]] = np.nan
        
ibex_historical_data = pd.DataFrame(ibex_historical_data)

table = pa.Table.from_pandas(ibex_historical_data)
buf = BytesIO()
pq.write_table(table, buf)

s3.Object(bucket_name, full_path).put(Body=buf.getvalue())

Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2000-07-03  40.500000  42.250000  39.500000  41.650002  -1.961556    46596
2000-07-04  42.000000  42.150002  40.400002  40.599998  -1.912106    83363
2000-07-05  40.610001  41.099998  39.900002  40.000000  -1.883848   202897
2000-07-06  40.099998  41.299999  40.099998  40.660000  -1.914931   196208
2000-07-10  41.299999  41.599998  41.099998  41.240002  -1.942247   169478
...               ...        ...        ...        ...        ...      ...
2001-01-08  39.900002  41.099998  39.500000  41.049999  -1.933298   146029
2001-01-09  41.099998  41.750000  39.709999  40.150002  -1.890912   148356
2001-01-10  40.250000  40.439999  39.900002  40.400002  -1.902686   110611
2001-01-11  40.470001  40.799999  40.290001  40.650002  -1.914460  1063247
2001-01-12  40.360001  41.200001  40.099998  41.000000  -1.930944   191471

[131 rows x 6 columns]
p

C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2000-01-03  19.700001  19.750000  18.879999  19.010000  -4.884562  3615680
2000-01-04  18.879999  18.879999  18.200001  18.500000  -4.753520  4048543
2000-01-05  18.200001  18.680000  18.129999  18.639999  -4.789490  4688995
2000-01-07  18.900000  19.350000  18.860001  19.309999  -4.961645  4027102
2000-01-10  19.549999  19.549999  19.059999  19.309999  -4.961645  2533020
...               ...        ...        ...        ...        ...      ...
2000-06-26  21.280001  21.320000  20.850000  21.250000  -5.460122  5896996
2000-06-27  21.100000  21.340000  20.809999  20.850000  -5.357343  3900422
2000-06-28  20.850000  20.900000  20.500000  20.690001  -5.316232  7792265
2000-06-29  20.690001  20.770000  20.090000  20.160000  -5.180052  4060382
2000-06-30  20.250000  20.500000  20.090000  20.290001  -5.213454  3817463

[124 rows x 6 columns]
p

...

             Open   High    Low  Close  Adj Close    Volume
Date                                                       
2001-07-02  3.758  3.840  3.728  3.820  -0.136871   8634625
2001-07-03  3.772  3.822  3.772  3.820  -0.136871  20450495
2001-07-04  3.820  3.830  3.766  3.796  -0.136011   4993255
2001-07-05  3.796  3.810  3.730  3.730  -0.133647  16127145
2001-07-06  3.760  3.760  3.700  3.700  -0.132572   6726610
...           ...    ...    ...    ...        ...       ...
2002-09-18  4.062  4.098  4.000  4.052  -0.146037   4711710
2002-09-19  4.022  4.150  3.960  4.020  -0.144884   9985435
2002-09-20  4.076  4.350  3.966  4.280  -0.154255  16214830
2002-09-23  4.210  4.228  4.082  4.200  -0.151371   8471515
2002-09-24  4.200  4.260  4.106  4.200  -0.151371   8724120

[308 rows x 6 columns]
preprocessed/yfinance/ITX.parquet
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empt

C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


C:\Users\aleja\AppData\Local\Temp\ipykernel_28836\1811459781.py:23: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  ibex_historical_data = pd.DataFrame(ibex_historical_data)


ValueError: cannot reindex on an axis with duplicate labels

In [23]:
failed_loads = ibex_historical_data.columns[ibex_historical_data.isnull().values.all(axis=0)]
failed_loads, len(failed_loads)

(Index(['A3TV', 'ABG.P', 'ABG.P_1', 'ABG', 'ACR', 'ACX', 'AGR', 'AGS', 'AGS_1',
        'AGS_2',
        ...
        'UNI', 'URA', 'URB', 'VAL', 'VAL_1', 'VDR', 'VIS', 'VIS_1', 'ZEL',
        'ZOT'],
       dtype='object', length=101),
 101)

In [12]:
ibex_historical_data

,A3TV,ABE,ABG.P,ABG.P_1,ABG,ACE,ACR,ACS,ACX,ACX_1,...,URB,VAL,VAL_1,VDR,VIS,VIS_1,VIS_2,VIS_3,ZEL,ZOT
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,NaN,3.655523,NaN,NaN,NaN,3.637537,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,3.573376,NaN,NaN,NaN,3.636639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,3.532304,NaN,NaN,NaN,3.516951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.516951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,3.573376,NaN,NaN,NaN,3.783944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.309999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.740000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.809999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
